In [ ]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.compose import TransformedTargetRegressor
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, OPTICS, Birch, MeanShift, SpectralClustering, AffinityPropagation, FeatureAgglomeration

# import regressors
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor, HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor, PassiveAggressiveRegressor, Perceptron, RidgeClassifier, LogisticRegression
from sklearn.linear_model import Lasso, ElasticNet, Lars, BayesianRidge, ARDRegression, OrthogonalMatchingPursuit, HuberRegressor, TheilSenRegressor, RANSACRegressor
from sklearn.linear_model import LassoLars, LassoLarsIC
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV, LarsCV, OrthogonalMatchingPursuitCV, LassoLarsCV, BayesianRidge, LinearRegression

# pandas deactivate future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

SUBMIT = True
USE_ORIGINAL = True
SEED = 15
SAMPLE = 1

train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')
orig = pd.read_csv('datasets/cubic_zirconia.csv')

for i, df in enumerate([train, test, orig]):
    df.drop(['id'], axis=1, inplace=True)
    if not SUBMIT:
        df.drop_duplicates(inplace=True)
    # df['dataset'] = i

# Define test set
if not SUBMIT:
    train, test = train_test_split(train, test_size=0.2, random_state=SEED) 

if USE_ORIGINAL:
    train = pd.concat([train, orig], axis=0)
    train.reset_index(inplace=True, drop=True)

# Sampling for faster training
if SAMPLE < 1:
    train = train.sample(frac=SAMPLE, random_state=SEED)

del orig

# set training data
X_train = train.copy()
y_train = X_train.pop('price')
X_test = test.copy()

if not SUBMIT:
    y_test = X_test.pop('price')
else:
    y_test = None
    
base_cols = X_train.columns

# transform categorical features
def transform_categorical(df):
    df['cut'] = df['cut'].map({'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4})
    df['color'] = df['color'].map({'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6})
    df['clarity'] = df['clarity'].map({'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7})
    return df

def remove_outliers(df):
    # Drop extreme values
    min = 2
    max = 20
    df = df[(df['x'] < max) & (df['y'] < max) & (df['z'] < max)]
    df = df[(df['x'] > min) & (df['y'] > min) & (df['z'] > min)]
    return df

def add_volume_ratio(df):
    # df['volume_ratio1'] = (df['x'] * df['y']) / (df['z'] * df['z'])
    df['volume_ratio2'] = (df['x'] * df['z']) / (df['y'] * df['y'])
    df['volume_ratio3'] = (df['y'] * df['z']) / (df['x'] * df['x'])
    # df['volume_ratio4'] = (df['x']) / (df['z'])
    # df['volume_ratio5'] = (df['y']) / (df['z'])
    df['volume_ratio6'] = (df['x'] * df['z']) / (df['y'] * df['z'])  # will set nan if z is nan
    # df['volume_ratio7'] = (df['x'] + df['y']) / df['z']
    # df['volume_ratio8'] = (df['x'] + df['z']) / df['y']
    # df['volume_ratio9'] = (df['y'] + df['z']) / df['x']
    # df['volume_ratio10'] = (df['x'] * df['y'] * df['z']) / (df['x'].mean() * df['y'].mean() * df['z'].mean())
    # df['volume_ratio11'] = (df['x'] * df['y'] * df['z']) / (df['x'].max() * df['y'].max() * df['z'].max())
    # df['volume_ratio12'] = (df['x'] * df['y'] * df['z']) / (df['x'].min() * df['y'].min() * df['z'].min())
    # df['volume_ratio13'] = (df['x'] * df['y'] * df['z']) / (df['x'].median() * df['y'].median() * df['z'].median())
    # df['volume_ratio14'] = (df['x'] * df['y'] * df['z']) / (df['x'].std() * df['y'].std() * df['z'].std())
    return df

def feature_engineering(df):
    df["volume"] = df["x"] * df["y"] * df["z"]
    df["surface_area"] = 2 * (df["x"] * df["y"] + df["y"] * df["z"] + df["z"] * df["x"])
    df["aspect_ratio_xy"] = df["x"] / df["y"]
    df["aspect_ratio_yz"] = df["y"] / df["z"]
    df["aspect_ratio_zx"] = df["z"] / df["x"]
    df["diagonal_distance"] = np.sqrt(df["x"] ** 2 + df["y"] ** 2 + df["z"] ** 2)
    # df["relative_height"] = (df["z"] - df["z"].min()) / (df["z"].max() - df["z"].min())
    # df["relative_position"] = (df["x"] + df["y"] + df["z"]) / (df["x"] + df["y"] + df["z"]).sum()
    # df["volume_ratio"] = df["x"] * df["y"] * df["z"] / (df["x"].mean() * df["y"].mean() * df["z"].mean())
    # df["length_ratio"] = df["x"] / df["x"].mean()
    # df["width_ratio"] = df["y"] / df["y"].mean()
    # df["height_ratio"] = df["z"] / df["z"].mean()
    df["sphericity"] = 1.4641 * (6 * df["volume"])**(2/3) / df["surface_area"]
    df["compactness"] = df["volume"]**(1/3) / df["x"]
    df['density'] = df['carat'] / df['volume']
    df['table_percentage'] = (df['table'] / ((df['x'] + df['y']) / 2)) * 100
    df['depth_percentage'] = (df['depth'] / ((df['x'] + df['y']) / 2)) * 100
    df['symmetry'] = (abs(df['x'] - df['z']) + abs(df['y'] - df['z'])) / (df['x'] + df['y'] + df['z'])
    df['surface_area'] = 2 * ((df['x'] * df['y']) + (df['x'] * df['z']) + (df['y'] * df['z']))
    df['depth_to_table_ratio'] = df['depth'] / df['table']
    df['girdle_diameter'] = 100 * df['z'] / df['depth']
    df['girdle_thickness'] = 100 * df['z'] / df['table']
    df['girdle_ratio'] = df['girdle_diameter'] / df['girdle_thickness']
    return df

def target_transform(serie):
    serie = np.log1p(serie)
    return serie

def inverse_target_transform(serie):
    serie = np.expm1(serie)
    return serie

def set_categorical(df):
    df['cut'] = df['cut'].astype('category')
    df['color'] = df['color'].astype('category')
    df['clarity'] = df['clarity'].astype('category')
    return df

def add_girdle_parameters(df):
    df['girdle_diameter'] = 100 * df['z'] / df['depth']
    df['girdle_thickness'] = 100 * df['z'] / df['table']
    df['girdle_ratio'] = df['girdle_diameter'] / df['girdle_thickness']
    return df

def impute_x_y_z(df):
    df['is_imputed'] = df.isna().any(axis=1).astype(int)
    df['girdle_diameter'].fillna((df['x'] + df['y']) / 2, inplace=True)
    df['x'].fillna(2*df['girdle_diameter'] - df['y'], inplace=True)
    df['y'].fillna(2*df['girdle_diameter'] - df['x'], inplace=True)
    df['z'].fillna(df['girdle_diameter'] * df['depth'] / 100, inplace=True)
    df = add_girdle_parameters(df)
    return df

def set_nan(df):
    for col in ['x', 'y', 'z']:
        df[col].replace(0, np.nan, inplace=True)
    return df

def drop_girdle_parameters(df):
    df.drop(['girdle_diameter', 'girdle_thickness', 'girdle_ratio'], axis=1, inplace=True)
    return df

# Make data preparation pipeline
def data_prepation(X_train, X_test):
    
    for df in [X_train, X_test]:
        # df = set_nan(df)
        df = transform_categorical(df)
        # df = set_categorical(df)
        # df = add_girdle_parameters(df)
        # df = impute_x_y_z(df)
        # df = drop_girdle_parameters(df)
        
    
    # imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # imputer = IterativeImputer(max_iter=10, random_state=0)
    # imputer = KNNImputer(n_neighbors=1, weights="uniform")
    # X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    # X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)
    
    # selected_cols = base_cols
    # selected_cols = ['surface_area', 'clarity', 'color', 'cut', 'carat', 'depth_percentage', 'depth', 'compactness', 'depth_to_table_ratio']
    
    # for df in [X_train, X_test]:
    #     df = add_volume_ratio(df)
        # df = feature_engineering(df)
        
        # df.fillna(0, inplace=True)
        # df.replace([np.inf, -np.inf], 0, inplace=True)
        # df.dropna(inplace=True)
        # df.drop([col for col in df.columns if col not in selected_cols], axis=1, inplace=True)
        
    # Scaling
    # scaler = PowerTransformer()
    # X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    # X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    
    # Clustering features
    # model = KMeans(n_clusters=20, random_state=42)
    # X_train['cluster'] = model.fit_predict(X_train)
    # X_test['cluster'] = model.predict(X_test)
    
    return X_train, X_test
            
data_prep_has_fit_method = False

if not data_prep_has_fit_method:
    X_train, X_test = data_prepation(X_train, X_test)
    X_train_prep, X_test_prep = X_train.copy(), X_test.copy()
else:
    X_train_prep, X_test_prep = data_prepation(X_train.copy(), X_test.copy())

   
# X_train_prep, X_test_prep = data_prepation(X_train.copy(), X_test.copy())
# pd.DataFrame(X_train_prep.isna().sum(), columns=['train']).join(pd.DataFrame(X_test_prep.isna().sum(), columns=['test']))
X_train

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# Set categorical features for catboost
cat_features = [col for col in X_train_prep.columns if X_train_prep[col].dtype == 'category']

regressors = {
    'LGBMRegressor1': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt'),
    # 'LGBMRegressor2': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='dart'),
    'LGBMRegressor3': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='goss'),
    # 'LGBMRegressor4': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='rf', subsample=.632, subsample_freq=1),
    'LGBMRegressor5': LGBMRegressor(random_state=SEED, n_jobs=-1, class_weight='balanced'),
    'LGBMRegressor6': LGBMRegressor(random_state=SEED, n_jobs=-1, subsample=0.7),
    'LGBMRegressor7': LGBMRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=0.7),
    'LGBMRegressor8': LGBMRegressor(random_state=SEED, n_jobs=-1, subsample=0.7, colsample_bytree=0.7),
    # 'LGBMRegressor9': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='dart', colsample_bytree=0.7),
    'LGBMRegressor10': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt', num_leaves=48, max_depth=14, learning_rate=0.08, n_estimators=240),
    'LGBMRegressor11': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt', num_leaves=48, max_depth=14, learning_rate=0.08, n_estimators=240, subsample=0.7, colsample_bytree=0.6),
    'XGBRegressor1': XGBRegressor(random_state=SEED, n_jobs=-1),
    'XGBRegressor2': XGBRegressor(random_state=SEED, n_jobs=-1, booster='dart'),
    'XGBRegressor3': XGBRegressor(random_state=SEED, n_jobs=-1, booster='gblinear'),
    'XGBRegressor4': XGBRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=0.7),
    'XGBRegressor5': XGBRegressor(random_state=SEED, n_jobs=-1, subsample=0.7),
    'XGBRegressor6': XGBRegressor(random_state=SEED, 
                                  n_jobs=-1, 
                                  learning_rate=0.055, 
                                  n_estimators=200, 
                                  max_depth=8, 
                                  min_child_weight=1, 
                                  gamma=0.07, 
                                  colsample_bytree=0.67, 
                                  colsample_bylevel=0.67, 
                                  colsample_bynode=0.8,
                                  subsample=0.7, 
                                  objective='reg:squarederror'),
    'XGBRegressor7': XGBRegressor(random_state=SEED, n_jobs=-1, objective='reg:squarederror'),
    'XGBRandomForestRegressor': XGBRFRegressor(random_state=SEED, n_jobs=-1),
    'CatBoostRegressor': CatBoostRegressor(random_state=SEED, silent=True), # Promising but fails on the cv
    'HistGradientBoostingRegressor': HistGradientBoostingRegressor(random_state=SEED),
    'HistGradientBoostingRegressor2': HistGradientBoostingRegressor(random_state=SEED, max_iter=200, 
                                                                    max_depth=6, learning_rate=0.1, 
                                                                    l2_regularization=0.0006754828207682132, max_leaf_nodes=42, 
                                                                    min_samples_leaf=9, max_bins=255),
    'HistGradientBoostingRegressor3': HistGradientBoostingRegressor(random_state=SEED, max_iter=1000, 
                                                                    max_depth=10, learning_rate=0.1, 
                                                                    l2_regularization=0.1, max_leaf_nodes=100, 
                                                                    min_samples_leaf=20, max_bins=255),
    # 'RandomForestRegressor': RandomForestRegressor(random_state=SEED, n_jobs=-1),
    # 'ExtraTreesRegressor': ExtraTreesRegressor(random_state=SEED, n_jobs=-1),
    # 'AdaBoostRegressor': AdaBoostRegressor(random_state=SEED),
    # 'GradientBoostingRegressor': GradientBoostingRegressor(random_state=SEED),
    # 'BaggingRegressor': BaggingRegressor(random_state=SEED, n_jobs=-1),
    # 'KNeighborsRegressor': KNeighborsRegressor(n_jobs=-1),
    # 'DecisionTreeRegressor': DecisionTreeRegressor(random_state=SEED),
    # 'GaussianProcessRegressor': GaussianProcessRegressor(random_state=SEED),
    # 'MLPRegressor1': MLPRegressor(random_state=SEED, max_iter=1000, activation='relu', solver='adam'),
    # 'MLPRegressor2': MLPRegressor(random_state=SEED, max_iter=1000, activation='relu', solver='lbfgs'), # promising but long to train
    # 'MLPRegressor3': MLPRegressor(random_state=SEED, max_iter=5000, activation='tanh', solver='adam'),
    # 'MLPRegressor4': MLPRegressor(random_state=SEED, max_iter=1000, activation='tanh', solver='lbfgs'),  # promising but long to train
    # 'MLPRegressor5': MLPRegressor(random_state=SEED, max_iter=1000, activation='logistic', solver='adam'),
    # 'MLPRegressor6': MLPRegressor(random_state=SEED, max_iter=1000, activation='logistic', solver='lbfgs'),
    # 'MLPRegressor7': MLPRegressor(random_state=SEED, max_iter=1000, activation='identity', solver='adam'),
    # 'MLPRegressor8': MLPRegressor(random_state=SEED, max_iter=1000, activation='identity', solver='lbfgs'),
    # 'Ridge': Ridge(random_state=SEED),
    # 'SGDRegressor': SGDRegressor(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'PassiveAggressiveRegressor': PassiveAggressiveRegressor(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'Perceptron': Perceptron(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'LinearRegression': LinearRegression(),
    # 'Lasso': Lasso(random_state=SEED),
    # 'ElasticNet': ElasticNet(random_state=SEED, max_iter=1e6),
    # 'HuberRegressor': HuberRegressor(max_iter=1000),
    # 'BayesianRidge': BayesianRidge(),
    # 'ARDRegression': ARDRegression(),
    # 'TheilSenRegressor': TheilSenRegressor(random_state=SEED),
    # 'RANSACRegressor': RANSACRegressor(random_state=SEED),
    # 'OrthogonalMatchingPursuit': OrthogonalMatchingPursuit(normalize=False),
    # 'Lars': Lars(),
    # 'LassoLars': LassoLars(),
    # 'LassoLarsIC': LassoLarsIC(normalize=False),
    # 'StackingRegressor': StackingRegressor(
    #         estimators=[
    #             ('LGBMRegressor11', LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt', num_leaves=48, 
    #                                             max_depth=14, learning_rate=0.08, n_estimators=240, subsample=0.7, colsample_bytree=0.6)),
    #             ('XGBRegressor6', XGBRegressor(random_state=SEED, n_jobs=-1, learning_rate=0.055, n_estimators=200,  
    #                                         max_depth=8,  min_child_weight=1, gamma=0.07,  colsample_bytree=0.67, 
    #                                         colsample_bylevel=0.67, colsample_bynode=0.8, subsample=0.7, 
    #                                         objective='reg:squarederror')),
    #             ('CatBoostRegressor', CatBoostRegressor(random_state=SEED, silent=True, cat_features=cat_features)), # Promising but fails on the cv
    #             # ('ExtraTreesRegressor', ExtraTreesRegressor(random_state=SEED, n_jobs=-1))
    #             ], 
    #         final_estimator=Ridge(random_state=SEED),
    #         cv=cv,
    #         # n_jobs=-1,
    #         verbose=1
    #         )
}

for model_name, regressor in regressors.items():
    t0 = time.time()
    scores = []
    feature_importances = pd.DataFrame()
    # ttr = TransformedTargetRegressor(regressor=regressor, func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)
    ttr = regressor
    
    for i, (train_index, test_index) in tqdm(enumerate(cv.split(X_train))):
        
        X_train_cv, X_test_cv = X_train.iloc[train_index].copy(), X_train.iloc[test_index].copy()
        y_train_cv, y_test_cv = y_train.iloc[train_index].copy(), y_train.iloc[test_index].copy()
        
        if data_prep_has_fit_method:
            X_train_cv, X_test_cv = data_prepation(X_train_cv, X_test_cv)

        # Models that need scaling and no missing value
        if model_name in ['MLPRegressor1', 'MLPRegressor2', 'MLPRegressor3', 'MLPRegressor4', 'MLPRegressor5', 'MLPRegressor6', 'MLPRegressor7', 'MLPRegressor8', 'SGDRegressor', 'PassiveAggressiveRegressor', 'Perceptron', 'Ridge', 'Lasso', 'ElasticNet', 'HuberRegressor', 'BayesianRidge', 'ARDRegression', 'TheilSenRegressor', 'RANSACRegressor', 'OrthogonalMatchingPursuit', 'Lars', 'LassoLars', 'LassoLarsIC']:
            scaler = StandardScaler()
            X_train_cv = pd.DataFrame(scaler.fit_transform(X_train_cv), columns=X_train_cv.columns)
            X_test_cv = pd.DataFrame(scaler.transform(X_test_cv), columns=X_test_cv.columns)

            imputer = KNNImputer(n_neighbors=5)
            X_train_cv = pd.DataFrame(imputer.fit_transform(X_train_cv), columns=X_train_cv.columns)
            X_test_cv = pd.DataFrame(imputer.transform(X_test_cv), columns=X_test_cv.columns)

            
        ttr.fit(X_train_cv, y_train_cv)        
        y_pred = ttr.predict(X_test_cv)
        score_eval = mean_squared_error(y_test_cv, y_pred, squared=False)
        scores.append(score_eval)
        
        try:
            feature_importance = pd.Series(ttr.regressor_.feature_importances_, index=X_train_cv.columns, name=f'fold{i}')
        except:
            try:
                feature_importance = pd.Series(ttr.regressor_.coef_, index=X_train_cv.columns, name=f'fold{i}')
            except:
                feature_importance = pd.Series(np.zeros(X_train_cv.shape[1]), index=X_train_cv.columns, name=f'fold{i}')
        feature_importances = pd.concat([feature_importances, feature_importance], axis=1)
    
    feature_importances['mean'] = feature_importances.mean(axis=1)
    
    ttr.fit(X_train_prep, y_train)
    y_pred = ttr.predict(X_test_prep)
    
    if not SUBMIT:
        score_eval = mean_squared_error(y_test, y_pred, squared=False)
    
    print(f'{model_name}: {np.mean(scores):.4f} ± {np.std(scores):.4f}, Time: {time.time() - t0:.2f} seconds, RMSE: {score_eval:.4f}')
    # print(feature_importances.sort_values('mean', ascending=False))

# Simple stacking with sklearn with target transformation

In [3]:
# Simple stacking with sklearn with target transformation
regressors = [
    ('LGBMRegressor11', LGBMRegressor(random_state=SEED, n_jobs=1, boosting_type='gbdt', num_leaves=48, 
                                     max_depth=14, learning_rate=0.08, n_estimators=240, subsample=0.7, colsample_bytree=0.6)),
    ('XGBRegressor6', XGBRegressor(random_state=SEED, n_jobs=-1, learning_rate=0.055, n_estimators=200,  
                                  max_depth=8,  min_child_weight=1, gamma=0.07,  colsample_bytree=0.67, 
                                  colsample_bylevel=0.67, colsample_bynode=0.8, subsample=0.7, 
                                  objective='reg:squarederror')),
    ('CatBoostRegressor', CatBoostRegressor(random_state=SEED, silent=True)),
    ('HistGradientBoostingRegressor2', HistGradientBoostingRegressor(random_state=SEED, max_iter=1000, 
                                                                    max_depth=10, learning_rate=0.1, 
                                                                    l2_regularization=0.1, max_leaf_nodes=100, 
                                                                    min_samples_leaf=20, max_bins=255)),
    
]

model = StackingRegressor(
    estimators=regressors,
    final_estimator=LarsCV(cv=cv, max_iter=10000, n_jobs=-1),
    cv=cv,
    n_jobs=-1,
    verbose=1,
    )

y_train_transformed = target_transform(y_train)

def scorer(estimator, X, y):
    y_pred = inverse_target_transform(estimator.predict(X))
    return -mean_squared_error(inverse_target_transform(y), y_pred, squared=False)

scores = cross_val_score(model, X_train_prep, y_train_transformed, cv=cv, scoring=scorer, n_jobs=-1)
print(f'{np.mean(scores):.4f} ± {np.std(scores):.4f}')

model.fit(X_train_prep, y_train_transformed)
y_pred_StackingRegressor = inverse_target_transform(model.predict(X_test_prep))

# Save predictions
sub = pd.read_csv('submissions/sample_submission.csv')
sub['price'] = y_pred_StackingRegressor
now = time.strftime("%Y-%m-%d %H_%M_%S")
sub.to_csv(f'submissions/submission_StackingRegressor{now}.csv', index=False)

# Simple stacking with sklearn without target transformation

In [ ]:
# Simple stacking with sklearn without target transformation
regressors = [
    ('LGBMRegressor11', LGBMRegressor(random_state=SEED, n_jobs=1, boosting_type='gbdt', num_leaves=48, 
                                     max_depth=14, learning_rate=0.08, n_estimators=240, subsample=0.7, colsample_bytree=0.6)),
    ('XGBRegressor6', XGBRegressor(random_state=SEED, n_jobs=-1, learning_rate=0.055, n_estimators=200,  
                                  max_depth=8,  min_child_weight=1, gamma=0.07,  colsample_bytree=0.67, 
                                  colsample_bylevel=0.67, colsample_bynode=0.8, subsample=0.7, 
                                  objective='reg:squarederror')),
    ('CatBoostRegressor', CatBoostRegressor(random_state=SEED, silent=True)),
    ('HistGradientBoostingRegressor2', HistGradientBoostingRegressor(random_state=SEED, max_iter=1000, 
                                                                    max_depth=10, learning_rate=0.1, 
                                                                    l2_regularization=0.1, max_leaf_nodes=100, 
                                                                    min_samples_leaf=20, max_bins=255)),
    
]

model = StackingRegressor(
    estimators=regressors,
    final_estimator=LarsCV(cv=cv, max_iter=10000, n_jobs=-1),
    cv=cv,
    n_jobs=-1,
    verbose=1,
    )

model.fit(X_train_prep, y_train)
y_pred_StackingRegressor = model.predict(X_test_prep)

# Save predictions
sub = pd.read_csv('submissions/sample_submission.csv')
sub['price'] = y_pred_StackingRegressor
now = time.strftime("%Y-%m-%d %H_%M_%S")
sub.to_csv(f'submissions/submission_StackingRegressor{now}.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import PredictionErrorDisplay

def plot_predictions_errors(y, y_pred, title):
    fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
    PredictionErrorDisplay.from_predictions(
        y,
        y_pred=y_pred,
        kind="actual_vs_predicted",
        subsample=100,
        ax=axs[0],
        random_state=0,
    )
    axs[0].set_title("Actual vs. Predicted values")
    PredictionErrorDisplay.from_predictions(
        y,
        y_pred=y_pred,
        kind="residual_vs_predicted",
        subsample=1000,
        ax=axs[1],
        random_state=SEED,
    )
    axs[1].set_title("Residuals vs. Predicted Values")
    fig.suptitle(title)
    plt.tight_layout()
    plt.show()

# Custom stacking for cross validation

In [ ]:
# CV stacking: 
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

regressors = {
    'LGBMRegressor11': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt', num_leaves=48, 
                                     max_depth=14, learning_rate=0.08, n_estimators=240, subsample=0.7, colsample_bytree=0.6),
    'XGBRegressor6': XGBRegressor(random_state=SEED, n_jobs=-1, learning_rate=0.055, n_estimators=200, 
                                  max_depth=8, min_child_weight=1, gamma=0.07, colsample_bytree=0.67, 
                                  colsample_bylevel=0.67, colsample_bynode=0.8, subsample=0.7, objective='reg:squarederror'),
    'CatBoostRegressor': CatBoostRegressor(random_state=SEED, silent=True), # Promising but fails on the cv
    'HistGradientBoostingRegressor3': HistGradientBoostingRegressor(random_state=SEED, max_iter=1000, 
                                                                    max_depth=10, learning_rate=0.1, 
                                                                    l2_regularization=0.1, max_leaf_nodes=100, 
                                                                    min_samples_leaf=20, max_bins=255),
}

meta_regressors = [
    ('LinearRegression', LinearRegression()),
    ('RidgeCV', RidgeCV(alphas=np.logspace(-3, 3, 13), cv=cv)),
    ('ElasticNetCV', ElasticNetCV(alphas=np.logspace(-3, 3, 13), cv=cv, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=100000)),
    # ('LassoCV', LassoCV(alphas=np.logspace(-3, 3, 13), cv=cv, max_iter=100000)),  # = ElasticNetCV with l1_ratio=1
    # ('LarsCV', LarsCV(cv=cv, max_iter=100000, n_jobs=-1)),
    # ('OrthogonalMatchingPursuitCV', OrthogonalMatchingPursuitCV(cv=cv, n_jobs=-1)),
    # ('LassoLarsCV', LassoLarsCV(cv=cv, max_iter=10000, n_jobs=-1)),
    # ('BayesianRidge', BayesianRidge()),
]

FIT_REGRESSORS = False
DISPLAY_REGRESSOR_RESULTS = True
PLOT_ERRORS = False
TARGET_TRANSFORMATION = False

if FIT_REGRESSORS:
    # Store out of fold predictions for meta learner
    X_meta_trains = {}  # Dict of datasets used for meta learner training
    X_meta_hold_outs = {}  # Dict of datasets used for meta learner validation
    X_meta_tests = {}  # Dict of datasets used for meta learner testing

for i, (train_index, hold_out_index) in enumerate(cv.split(X_train_prep)):
    t0 = time.time()
    print(f'Fold {i+1} of {cv.get_n_splits()}')
    X_train_cv, X_hold_out = X_train_prep.iloc[train_index].copy(), X_train_prep.iloc[hold_out_index].copy()
    y_train_cv, y_hold_out = y_train.iloc[train_index].copy(), y_train.iloc[hold_out_index].copy()
    
    X_meta_train = pd.DataFrame(index=train_index, columns=[name for name, _ in regressors.items()])
    X_meta_hold_out = pd.DataFrame(index=hold_out_index, columns=[name for name, _ in regressors.items()])
    X_meta_test = pd.DataFrame(index=X_test_prep.index, columns=[name for name, _ in regressors.items()])
    
    if FIT_REGRESSORS:
        # for name, regressor in regressors:
        for name, regressor in regressors.items():
            print(f'Fitting {name} ...')
            if TARGET_TRANSFORMATION:
                ttr = TransformedTargetRegressor(regressor=regressor, func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)
            else:
                ttr = regressor
            
            if name == 'CatBoostRegressor':
                X_meta_train[name] = cross_val_predict(ttr, X_train_cv, y_train_cv, cv=cv, verbose=0)  # CatBoostRegressor fails on n_jobs=-1
            else:
                X_meta_train[name] = cross_val_predict(ttr, X_train_cv, y_train_cv, cv=cv, n_jobs=-1, verbose=0)
            
            # fit the model on the full cv training set
            ttr.fit(X_train_cv, y_train_cv)
            X_meta_hold_out[name] = ttr.predict(X_hold_out)
            X_meta_test[name] = ttr.predict(X_test_prep)

            if DISPLAY_REGRESSOR_RESULTS:
                print(f'Hold out score of {name}: {mean_squared_error(y_hold_out, X_meta_hold_out[name], squared=False):.4f}')
                if not SUBMIT:
                    print(f'Test score of {name}: {mean_squared_error(y_test, X_meta_test[name], squared=False):.4f}')

        # Store datasets for meta learner
        X_meta_trains[i] = X_meta_train.copy()
        X_meta_hold_outs[i] = X_meta_hold_out.copy()
        X_meta_tests[i] = X_meta_test.copy()
        
    # Transform the predictions of regressors with target transform
    if TARGET_TRANSFORMATION:  
        X_meta_train = target_transform(X_meta_trains[i])
        X_meta_hold_out = target_transform(X_meta_hold_outs[i])
        X_meta_test = target_transform(X_meta_tests[i])
    else:
        X_meta_train = X_meta_trains[i]
        X_meta_hold_out = X_meta_hold_outs[i]
        X_meta_test = X_meta_tests[i]

    for name, meta_regressor in meta_regressors:
        # Fit the final estimator on the hold out predictions
        if TARGET_TRANSFORMATION:
            meta_ttr = TransformedTargetRegressor(
                meta_regressor,
                func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)
        else:
            meta_ttr = meta_regressor
        
        meta_ttr.fit(X_meta_train, y_train_cv)
        y_hold_out_pred = meta_ttr.predict(X_meta_hold_out)
        y_test_pred = meta_ttr.predict(X_meta_test)
        
        if not SUBMIT:
            score_eval = mean_squared_error(y_test, y_test_pred, squared=False)
        else:
            score_eval = np.nan
        
        l1_ratio = getattr(meta_ttr.regressor_, 'l1_ratio_', np.nan) if TARGET_TRANSFORMATION else getattr(meta_ttr, 'l1_ratio_', np.nan)
        alpha = getattr(meta_ttr.regressor_, 'alpha_', np.nan) if TARGET_TRANSFORMATION else getattr(meta_ttr, 'alpha_', np.nan)
        coef = getattr(meta_ttr.regressor_, 'coef_', np.nan) if TARGET_TRANSFORMATION else getattr(meta_ttr, 'coef_', np.nan)
        intercept = getattr(meta_ttr.regressor_, 'intercept_', np.nan) if TARGET_TRANSFORMATION else getattr(meta_ttr, 'intercept_', np.nan)
        
        print(f'Meta regressor: {name}, RMSE hold out: {mean_squared_error(y_hold_out, y_hold_out_pred, squared=False)},',
              f'RMSE test: {score_eval:.4f}, fit time: {time.time() - t0:.2f} s,',
              f'Coefficients: {coef}, intercept: {intercept}, l1_ratio: {l1_ratio}, alpha: {alpha}', end='\n'
        )
        if PLOT_ERRORS:
            plot_predictions_errors(y_hold_out, y_hold_out_pred, 'Hold out')
            if not SUBMIT:
                plot_predictions_errors(y_test, y_test_pred, 'Test')

    print('-'*80, end='\n\n')

In [ ]:
# Save all dataframes of X_meta_trains and X_meta_hold_outs to csv seprarately
# for i in range(5):
#     X_meta_trains[i].to_csv(f'datasets/X_meta_trains_{i}.csv')
#     X_meta_hold_outs[i].to_csv(f'datasets/X_meta_hold_outs_{i}.csv')
#     X_meta_tests[i].to_csv(f'datasets/X_meta_tests_{i}.csv')

In [ ]:
# Load all dataframes of X_meta_trains and X_meta_hold_outs from csv seprarately
# X_meta_trains_2, X_meta_hold_outs_2 = {}, {}
# for i in range(5):
#     X_meta_trains_2[i] = pd.read_csv(f'datasets/X_meta_trains_{i}.csv', index_col=0)
#     X_meta_hold_outs_2[i] = pd.read_csv(f'datasets/X_meta_hold_outs_{i}.csv', index_col=0)
#     X_meta_tests_2[i] = pd.read_csv(f'datasets/X_meta_tests_{i}.csv', index_col=0)